In [1]:
import numpy as np 
import pandas as pd 
from datetime import date

In [2]:
ratios = [0,0.236, 0.382, 0.5 , 0.618, 0.786,1,1.414,1.618]
fiblevels = np.array(ratios)

In [3]:
def RSI(series, period=14):
    delta = series.diff().dropna()
    ups = delta * 0
    downs = ups.copy()
    ups[delta > 0] = delta[delta > 0]
    downs[delta < 0] = -delta[delta < 0]
    ups[ups.index[period-1]] = np.mean( ups[:period] ) #first value is sum of avg gains
    ups = ups.drop(ups.index[:(period-1)])
    downs[downs.index[period-1]] = np.mean( downs[:period] ) #first value is sum of avg losses
    downs = downs.drop(downs.index[:(period-1)])
    rs = ups.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() / \
         downs.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() 
    return 100 - 100 / (1 + rs)


# calculating Stoch RSI (gives the same values as TradingView)
# https://www.tradingview.com/wiki/Stochastic_RSI_(STOCH_RSI) 
def STRSIA(series, period=14, smoothK=3, smoothD=3):
    # Calculate RSI 
    delta = series.diff().dropna()
    ups = delta * 0
    downs = ups.copy()
    ups[delta > 0] = delta[delta > 0]
    downs[delta < 0] = -delta[delta < 0]
    ups[ups.index[period-1]] = np.mean( ups[:period] ) #first value is sum of avg gains
    ups = ups.drop(ups.index[:(period-1)])
    downs[downs.index[period-1]] = np.mean( downs[:period] ) #first value is sum of avg losses
    downs = downs.drop(downs.index[:(period-1)])
    rs = ups.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() / \
         downs.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() 
    rsi = 100 - 100 / (1 + rs)

    # Calculate StochRSI 
    stochrsi  = (rsi - rsi.rolling(period).min()) / (rsi.rolling(period).max() - rsi.rolling(period).min())
    stochrsi_K = stochrsi.rolling(smoothK).mean()
    stochrsi_D = stochrsi_K.rolling(smoothD).mean()
    return stochrsi, stochrsi_K, stochrsi_D

In [4]:
ratios

[0, 0.236, 0.382, 0.5, 0.618, 0.786, 1, 1.414, 1.618]

In [31]:
ada15 = pd.read_json('BN-main/BTC_USDT-1h.json')
ada15

,0,1,2,3,4,5
0,1609459200000,28923.63,29031.34,28690.17,28995.13,2311.811445
1,1609462800000,28995.13,29470.00,28960.35,29409.99,5403.068471
2,1609466400000,29410.00,29465.26,29120.03,29194.65,2384.231560
3,1609470000000,29195.25,29367.00,29150.02,29278.40,1461.345077
4,1609473600000,29278.41,29395.00,29029.40,29220.31,2038.046803
...,...,...,...,...,...,...
4653,1626235200000,31830.01,32015.98,31782.00,31930.12,1714.340347
4654,1626238800000,31932.74,31993.61,31655.83,31807.01,2234.653488
4655,1626242400000,31807.01,31942.35,31754.55,31835.47,1632.803810
4656,1626246000000,31829.03,32000.00,31825.56,31920.01,1452.029555


In [32]:
ada15.columns=['date','open','high','low','close','volume']
ada15

,date,open,high,low,close,volume
0,1609459200000,28923.63,29031.34,28690.17,28995.13,2311.811445
1,1609462800000,28995.13,29470.00,28960.35,29409.99,5403.068471
2,1609466400000,29410.00,29465.26,29120.03,29194.65,2384.231560
3,1609470000000,29195.25,29367.00,29150.02,29278.40,1461.345077
4,1609473600000,29278.41,29395.00,29029.40,29220.31,2038.046803
...,...,...,...,...,...,...
4653,1626235200000,31830.01,32015.98,31782.00,31930.12,1714.340347
4654,1626238800000,31932.74,31993.61,31655.83,31807.01,2234.653488
4655,1626242400000,31807.01,31942.35,31754.55,31835.47,1632.803810
4656,1626246000000,31829.03,32000.00,31825.56,31920.01,1452.029555


In [33]:
start = pd.to_datetime('2020-01-01')
ada15.date=pd.to_datetime(ada15['date'],unit='ms')
ada15


,date,open,high,low,close,volume
0,2021-01-01 00:00:00,28923.63,29031.34,28690.17,28995.13,2311.811445
1,2021-01-01 01:00:00,28995.13,29470.00,28960.35,29409.99,5403.068471
2,2021-01-01 02:00:00,29410.00,29465.26,29120.03,29194.65,2384.231560
3,2021-01-01 03:00:00,29195.25,29367.00,29150.02,29278.40,1461.345077
4,2021-01-01 04:00:00,29278.41,29395.00,29029.40,29220.31,2038.046803
...,...,...,...,...,...,...
4653,2021-07-14 04:00:00,31830.01,32015.98,31782.00,31930.12,1714.340347
4654,2021-07-14 05:00:00,31932.74,31993.61,31655.83,31807.01,2234.653488
4655,2021-07-14 06:00:00,31807.01,31942.35,31754.55,31835.47,1632.803810
4656,2021-07-14 07:00:00,31829.03,32000.00,31825.56,31920.01,1452.029555


In [34]:
start = pd.to_datetime('2021-04-01')
end = pd.to_datetime('2021-05-30')


ada2021 = ada15.loc[(ada15['date']> start)& (ada15['date']< end)].copy()
ada2021['RSI'] = RSI(ada2021['close'])
ada2021['RSI4']=RSI(ada2021['close'],56)
ada2021['RSI8']=RSI(ada2021['close'],112)
ada2021['STRSI'] = STRSIA(ada2021['close'])[0]
ada2021['STRSIK'] = STRSIA(ada2021['close'])[1]
ada2021['STRSID'] = STRSIA(ada2021['close'])[2]

ada2021['STRSI4'] = STRSIA(ada2021['close'],56,12,12)[0]
ada2021['STRSIK4'] = STRSIA(ada2021['close'],56,12,12)[1]
ada2021['STRSID4'] = STRSIA(ada2021['close'],56,12,12)[2]

ada2021['STRSI8'] = STRSIA(ada2021['close'],112,24,24)[0]
ada2021['STRSIK8'] = STRSIA(ada2021['close'],112,24,24)[1]
ada2021['STRSID8'] = STRSIA(ada2021['close'],112,24,24)[2]



ada2021

,date,open,high,low,close,volume,RSI,RSI4,RSI8,STRSI,STRSIK,STRSID,STRSI4,STRSIK4,STRSID4,STRSI8,STRSIK8,STRSID8
2159,2021-04-01 01:00:00,59221.95,59314.53,58931.43,59143.72,1518.676507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2160,2021-04-01 02:00:00,59139.74,59309.44,59014.90,59086.99,1220.337644,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2161,2021-04-01 03:00:00,59086.99,59165.28,58941.56,59119.02,1307.393155,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2162,2021-04-01 04:00:00,59119.02,59232.59,58622.00,58869.99,1695.238357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2163,2021-04-01 05:00:00,58869.99,58970.00,58660.65,58764.79,1547.236392,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3564,2021-05-29 19:00:00,34069.35,34329.59,33674.91,34025.43,4135.986849,35.260553,42.864288,45.164914,0.113152,0.074926,0.101846,0.034553,0.101780,0.230399,0.030612,0.157852,0.337036
3565,2021-05-29 20:00:00,34025.44,34494.93,33752.30,34234.18,4658.099088,38.056012,43.427646,45.411232,0.345260,0.190013,0.132950,0.085313,0.086748,0.214264,0.076234,0.153533,0.319852
3566,2021-05-29 21:00:00,34235.14,34742.93,34063.98,34650.08,3326.550872,43.308326,44.536994,45.899763,0.938734,0.465715,0.243552,0.185269,0.086672,0.197008,0.166718,0.159819,0.304396
3567,2021-05-29 22:00:00,34650.08,34651.73,34235.93,34467.84,2435.360360,41.642138,44.150733,45.718865,0.832861,0.705619,0.453782,0.162957,0.087191,0.178936,0.133212,0.165071,0.290547


In [35]:
#maxlevels = [13,13,13,13,13,13,13]
#maxlevels = np.array(maxlevels)
#maxlevels

In [36]:
#minlevels = [11,11,11,11,11,11,11]
#minlevels = np.array(minlevels)
#minlevels

In [37]:
ada2021

,date,open,high,low,close,volume,RSI,RSI4,RSI8,STRSI,STRSIK,STRSID,STRSI4,STRSIK4,STRSID4,STRSI8,STRSIK8,STRSID8
2159,2021-04-01 01:00:00,59221.95,59314.53,58931.43,59143.72,1518.676507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2160,2021-04-01 02:00:00,59139.74,59309.44,59014.90,59086.99,1220.337644,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2161,2021-04-01 03:00:00,59086.99,59165.28,58941.56,59119.02,1307.393155,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2162,2021-04-01 04:00:00,59119.02,59232.59,58622.00,58869.99,1695.238357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2163,2021-04-01 05:00:00,58869.99,58970.00,58660.65,58764.79,1547.236392,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3564,2021-05-29 19:00:00,34069.35,34329.59,33674.91,34025.43,4135.986849,35.260553,42.864288,45.164914,0.113152,0.074926,0.101846,0.034553,0.101780,0.230399,0.030612,0.157852,0.337036
3565,2021-05-29 20:00:00,34025.44,34494.93,33752.30,34234.18,4658.099088,38.056012,43.427646,45.411232,0.345260,0.190013,0.132950,0.085313,0.086748,0.214264,0.076234,0.153533,0.319852
3566,2021-05-29 21:00:00,34235.14,34742.93,34063.98,34650.08,3326.550872,43.308326,44.536994,45.899763,0.938734,0.465715,0.243552,0.185269,0.086672,0.197008,0.166718,0.159819,0.304396
3567,2021-05-29 22:00:00,34650.08,34651.73,34235.93,34467.84,2435.360360,41.642138,44.150733,45.718865,0.832861,0.705619,0.453782,0.162957,0.087191,0.178936,0.133212,0.165071,0.290547


In [38]:
high = ada2021.high
low = ada2021.low
maximums5 = high.rolling(5,5)
max5 = maximums5.max()
minimums5 = low.rolling(5,5)
min5 = minimums5.min()
maximums10 = high.rolling(10,10)
max10 = maximums10.max()
minimums10 = low.rolling(10,10)
min10 = minimums10.min()


In [39]:
len(max5)

1410

In [40]:

ada2021['max5'] = max5
ada2021['max10'] = max10
ada2021['min5'] = min5
ada2021['min10'] = min10

ada2021.tail()

,date,open,high,low,close,volume,RSI,RSI4,RSI8,STRSI,...,STRSI4,STRSIK4,STRSID4,STRSI8,STRSIK8,STRSID8,max5,max10,min5,min10
3564,2021-05-29 19:00:00,34069.35,34329.59,33674.91,34025.43,4135.986849,35.260553,42.864288,45.164914,0.113152,...,0.034553,0.101780,0.230399,0.030612,0.157852,0.337036,35049.00,35850.00,33632.76,33632.76
3565,2021-05-29 20:00:00,34025.44,34494.93,33752.30,34234.18,4658.099088,38.056012,43.427646,45.411232,0.345260,...,0.085313,0.086748,0.214264,0.076234,0.153533,0.319852,35049.00,35850.00,33632.76,33632.76
3566,2021-05-29 21:00:00,34235.14,34742.93,34063.98,34650.08,3326.550872,43.308326,44.536994,45.899763,0.938734,...,0.185269,0.086672,0.197008,0.166718,0.159819,0.304396,34856.96,35402.34,33632.76,33632.76
3567,2021-05-29 22:00:00,34650.08,34651.73,34235.93,34467.84,2435.360360,41.642138,44.150733,45.718865,0.832861,...,0.162957,0.087191,0.178936,0.133212,0.165071,0.290547,34742.93,35300.00,33632.76,33632.76
3568,2021-05-29 23:00:00,34468.95,35100.00,34137.09,34605.15,3780.594984,43.408765,44.519865,45.881019,1.000000,...,0.229423,0.090296,0.161060,0.163246,0.165591,0.277862,35100.00,35300.00,33674.91,33632.76


In [41]:
#maxnew = np.ones_like(ratios)
#maxnew

In [42]:
#addmin = mintest.tolist()
#addmax = maxtest.tolist()
#data = pd.DataFrame({"max":maxtest,"min":mintest},dtype='float')
#data = data.append(addmin,ignore_index=True)
#data1 = pd.DataFrame(maxtest)
#data2 = pd.DataFrame(mintest)
#data = pd.DataFrame(data1,data2)

In [43]:
#datamul = data.mul([2,1],axis='columns')
#datamul = data['max'].mul(ratios[7])

In [44]:
#datamul

In [45]:
#df = ada2021.loc[(ada2021['max10']>0)& (ada2021['min10']>0)]
df = ada2021
#len(df['date'])
average = df['max5']-df['min5']
avg = average.mean()
avg
df
#point = df.loc[(df['max5'].diff()>0)]
#pointm = df.loc[(df['min5'].diff()>0)]

df['support5']= df.min5.rolling(5,5).min()
df['support10']= df.min10.rolling(10,10).min()
df['resist5']= df.max5.rolling(5,5).max()
df['resist10']= df.max10.rolling(10,10).max()

#df['RSI'] = RSI(df['close'])
#df['STRSI4']=STRSIA(df['close'],56,12,12)
#df['STRSI8']=STRSIA(df['close'],112,24,24)


point = df[(df['max5'] > df['max5'].shift(1)) &  (df['max5'].shift(1) > df['max5'].shift(2))]
pointm = df[(df['min5'] < df['min5'].shift(1)) &  (df['min5'].shift(1) > df['min5'].shift(2))]
point
#ada2021
pointm

#for row in range(0,len(pointm)):
#    print(pointm['support5'])

,date,open,high,low,close,volume,RSI,RSI4,RSI8,STRSI,...,STRSIK8,STRSID8,max5,max10,min5,min10,support5,support10,resist5,resist10
2175,2021-04-01 17:00:00,58754.87,58850.62,58317.33,58501.63,2095.048857,30.686315,NaN,NaN,NaN,...,NaN,NaN,59263.30,59490.00,58317.33,58115.00,58115.00,NaN,59263.30,NaN
2182,2021-04-02 00:00:00,58720.45,58885.93,58455.10,58648.62,1936.788412,41.224841,NaN,NaN,NaN,...,NaN,NaN,59069.74,59263.30,58455.10,57935.45,57935.45,57935.45,59146.77,59490.00
2201,2021-04-02 19:00:00,59087.71,59087.71,58525.01,58850.00,2861.244906,39.661775,NaN,NaN,0.000000,...,NaN,NaN,59624.86,59660.30,58525.01,58525.01,58525.01,58525.01,59660.30,60200.00
2214,2021-04-03 08:00:00,59301.80,59791.72,58750.96,59028.65,3131.957995,45.233441,NaN,NaN,0.448547,...,NaN,NaN,59791.72,59791.72,58750.96,58733.97,58750.96,58428.57,59791.72,59791.72
2221,2021-04-03 15:00:00,59146.38,59248.18,58800.00,58829.99,2429.329924,41.159349,48.078443,NaN,0.000000,...,NaN,NaN,59461.28,59791.72,58800.00,58750.96,58750.96,58428.57,59791.72,59791.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3381,2021-05-22 04:00:00,36947.31,37219.45,35875.11,36304.18,7326.519005,38.424439,43.269087,43.709016,0.558881,...,0.766326,0.854242,38100.00,38100.00,35875.11,33488.00,33488.00,33488.00,38100.00,41789.40
3459,2021-05-25 10:00:00,37932.86,37937.15,36660.00,37103.24,8823.054266,43.978207,48.525145,46.966798,0.000000,...,0.905135,0.838821,39200.00,39550.00,36660.00,36660.00,36660.00,36660.00,39200.00,39920.00
3498,2021-05-27 01:00:00,38613.36,38714.27,37502.00,37625.31,4904.254570,38.632374,48.514017,47.741505,0.000000,...,0.882653,0.886812,39450.69,39450.69,37502.00,37502.00,37502.00,37502.00,39450.69,40841.00
3513,2021-05-27 16:00:00,39666.99,39721.28,38720.00,38825.44,6400.573744,48.584415,50.796907,48.996861,0.399874,...,0.826915,0.853398,40411.14,40411.14,38720.00,37826.08,37826.08,37134.27,40411.14,40411.14


In [68]:
#buypoints = df[(df['STRSI'] == 0) &  (df['STRSI4'] == 0) & (df['STRSI8'] == 0) & (df['close'] <  df['support10'].shift(1))]
buypoints = df[(df['STRSI'] == 0) &  (df['STRSI4'] == 0) & (df['STRSI8'] == 0)]
buypoints

,date,open,high,low,close,volume,RSI,RSI4,RSI8,STRSI,...,STRSIK8,STRSID8,max5,max10,min5,min10,support5,support10,resist5,resist10
2524,2021-04-16 06:00:00,61804.04,61804.04,61057.50,61455.69,5328.528357,32.461177,48.071468,50.860273,0.0,...,0.330071,0.433111,63211.70,63800.00,61057.50,61057.50,61057.50,61057.50,63520.61,63800.00
2525,2021-04-16 07:00:00,61455.69,61637.96,61025.00,61414.83,3183.223378,32.088771,47.934057,50.783987,0.0,...,0.310785,0.422490,63200.00,63520.61,61025.00,61025.00,61025.00,61025.00,63520.61,63800.00
2526,2021-04-16 08:00:00,61414.88,61607.83,61122.00,61191.91,2901.917131,30.062437,47.184833,50.368105,0.0,...,0.293615,0.411908,62961.81,63520.61,61025.00,61025.00,61025.00,61025.00,63520.61,63800.00
2527,2021-04-16 09:00:00,61193.23,61264.46,60400.00,60643.79,5366.020212,25.755748,45.407978,49.365142,0.0,...,0.276445,0.401221,62187.64,63520.61,60400.00,60400.00,60400.00,60400.00,63472.41,63800.00
2528,2021-04-16 10:00:00,60643.80,60969.50,60240.00,60247.54,4291.517046,23.171406,44.183288,48.658377,0.0,...,0.263539,0.390944,61804.04,63472.41,60240.00,60240.00,60240.00,60240.00,63211.70,63800.00
2559,2021-04-17 17:00:00,60377.01,60592.95,59637.12,59705.62,2492.896022,28.658551,43.247156,47.547704,0.0,...,0.212022,0.204777,61222.00,62458.28,59637.12,59637.12,59637.12,59637.12,62458.28,62506.05
2568,2021-04-18 02:00:00,59220.01,59481.79,58598.80,58804.21,4308.351015,27.608283,41.394381,46.193325,0.0,...,0.149914,0.205596,60819.95,60955.00,58598.80,58598.80,58598.80,58598.80,60955.00,62458.28
2569,2021-04-18 03:00:00,58804.21,58984.69,50931.30,55014.46,15734.664630,14.396314,33.204478,40.842343,0.0,...,0.134789,0.204534,60804.57,60955.00,50931.30,50931.30,50931.30,50931.30,60955.00,62458.28
2570,2021-04-18 04:00:00,55000.01,56033.92,53924.76,54829.24,12867.051882,14.042611,32.880749,40.610356,0.0,...,0.120828,0.202320,60499.00,60955.00,50931.30,50931.30,50931.30,50931.30,60955.00,61891.33
2691,2021-04-23 07:00:00,49051.48,49500.00,47700.00,47942.44,9913.986524,20.168929,34.965487,39.573062,0.0,...,0.459362,0.540928,50572.08,52234.74,47700.00,47700.00,47700.00,47700.00,52221.01,55280.00


In [69]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots


#Declare figure


fig = go.Figure()

fig = make_subplots(rows=8, cols=1,subplot_titles=("Price","gap","empty", "RSI 1 4 8", "Stoch 1", "Stoch 4","Stoch 8","Stoch 1 4 8"))
    

    
# buy points     
fig.add_trace(
    go.Scatter(x=buypoints.date, y=buypoints['close'],
        name="buy",
        text="buy",
        textposition="top center"),
    row=1, col=1
)    

    
# buy points     
fig.add_trace(
    go.Scatter(x=buypoints.date, y=buypoints['resist10'],
        name="resist10",
        text="resist10",
        textposition="top center"),
    row=1, col=1
)  
# resistance 10 


fig.add_trace(
    go.Scatter(x=df.date, y=df['RSI'],
        name="RSI",
        text="RSI",
        textposition="top center"),
    row=4, col=1
)

# support 10
fig.add_trace(
    go.Scatter(x=buypoints.date, y=buypoints['support10'],
        name="support10",
        text="support1010",
        textposition="top center"),
    row=1, col=1
)  

fig.add_trace(
    go.Scatter(x=df.date, y=df['RSI4'],name="RSI4"),
    row=4, col=1
)

fig.add_trace(
    go.Scatter(x=df.date, y=df['RSI8'],name="RSI8"),
    row=4, col=1
)

fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSI'],name="STRSI"),
    row=5, col=1
)
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSIK'],name="STRSIK"),
    row=5, col=1
)
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSID'],name="STRSID"),
    row=5, col=1
)
    
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSI4'],name="STRSI4"),
    row=6, col=1
)
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSIK4'],name="STRSIK4"),
    row=6, col=1
)
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSID4'],name="STRSID4"),
    row=6, col=1
)
    
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSI8'],name="STRSI8"),
    row=7, col=1
)
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSIK8'],name="STRSIK8"),
    row=7, col=1
)

fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSID8'],name="STRSIK8"),
    row=7, col=1
)


fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSI8'],name="STRSI8"),
    row=8, col=1
)
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSI4'],name="STRSI4"),
    row=8, col=1
)
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSI'],name="STRSI"),
    row=8, col=1
)


#add Traces

fig.add_trace(go.Candlestick(x=df.date,
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'], name = 'market data'),row=1,col=1)

#fig.add_trace(go.Scatter(x=df['date'], y=df['max10']))

#fig.add_trace(go.Scatter(x=df['date'], y=df['min10']))

#fig.add_trace(go.Scatter(x=point['date'], y=point['resist5']))

#fig.add_trace(go.Scatter(x=pointm['date'], y=pointm['support5']))


#fig.add_shape(type="line",
#    x0=pointm['date'], y0=pointm['support5'], x1=max(pointm['date']), y1=pointm['support5'],
#    line=dict(
#        color="LightSeaGreen",
#        width=4,
#        dash="dashdot",
#    ))
fig.update_shapes(dict(xref='x', yref='y'))

#for index in range(0,len(pointm)):
    #fig.add_hline(y=pointm['support5'])

#fig.add_trace(go.Scatter(x=point['date'], y=point['resist10']))

#fig.add_trace(go.Scatter(x=pointm['date'], y=pointm['support10']))
#for i in range(len(df['date'])):
    #fig.add_hline(y=df[i]['min10'], line_dash="dot",
                  #annotation_text="support", 
                  #annotation_position="bottom right")

#fig.add_hline(y=df['max10'], line_dash="dot",
#              annotation_text="resitance", 
#              annotation_position="bottom right")



fig.update_layout(height=3000, width=800, title_text="top sub plots")
# X and Y Axis Update
fig.update_xaxes(
    title = 'Date',rangeslider_visible=False
)

#fig.update_yaxes(
#    title = 'Share Price (USD)'
#)

#Show
fig.show()